In [15]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
import os
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Libraries imported.


In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content

In [17]:
soup = BeautifulSoup(results)
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [18]:
neighborhoods=pd.DataFrame(table_contents)
neighborhoods['Borough']=neighborhoods['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
neighborhoods.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [19]:
neighborhoods_geo = pd.read_csv('cordenadas.csv')
neighborhoods_geo.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
neighborhoods= neighborhoods.join(neighborhoods_geo.set_index('Postal Code'), on='Postal Code')


neighborhoods.head()



,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [21]:
address = 'toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto City are 43.6534817, -79.3839347.


In [22]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_Toronto)  
    
map_Toronto

In [23]:
North_York_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
North_York_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745905,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [24]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North Yorkare {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North Yorkare 43.7543263, -79.44911696639593.


In [25]:
# crear un mapa de Manhattan usando los valores de latitud y longitud
map_North_York = folium.Map(location=[latitude, longitude], zoom_start=11)

# añadir los marcadores al mapa
for lat, lng, label in zip(North_York_data['Latitude'], North_York_data['Longitude'], North_York_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_North_York)  
    
map_North_York

In [26]:
CLIENT_ID = 'OIUX4FNZEI4YT50ZURARYNWGZTNZWF100QUQS5EO3UJNYZEB' # su ID de Foursquare
CLIENT_SECRET = '40EAJOUOSIWTK4JMXV2DEK3PKJKRBRWFTIFIFHS212XDFPLW' # Secreto de Foursquare
VERSION = '20180604' # versión de la API de Foursquare
LIMIT = 50 # Un valor límite para la API de Foursquare

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OIUX4FNZEI4YT50ZURARYNWGZTNZWF100QUQS5EO3UJNYZEB
CLIENT_SECRET:40EAJOUOSIWTK4JMXV2DEK3PKJKRBRWFTIFIFHS212XDFPLW


In [27]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name


In [28]:
radius = 500 # define radius

 

url = "https://api.foursquare.com/v3/places/search?query=Toronto&ll=43.65107%2C-79.347015&radius=500&limit=50&session_token=fsq35wWzyrbQEe3PSyZW%2Fbg5afLh5cEC95plSGSFKQLTRXk%3D"

headers = {
    "accept": "application/json",
    "Authorization": "fsq35wWzyrbQEe3PSyZW/bg5afLh5cEC95plSGSFKQLTRXk="
}

response = requests.get(url, headers=headers)

print(response.text)



{"results":[{"fsq_id":"3f135697015a4f29a85bca53","categories":[{"id":11093,"name":"Mover","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/shops/realestate_","suffix":".png"}}],"chains":[],"distance":70,"geocodes":{"main":{"latitude":43.651708,"longitude":-79.346998},"roof":{"latitude":43.651708,"longitude":-79.346998}},"link":"/v3/places/3f135697015a4f29a85bca53","location":{"address":"9 Don Valley Pky","country":"CA","formatted_address":"9 Don Valley Pky, Toronto ON","locality":"Toronto","region":"ON"},"name":"Toronto Local Movers","related_places":{}},{"fsq_id":"4ad8f0abf964a520351621e3","categories":[{"id":11000,"name":"Business and Professional Services","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/default_","suffix":".png"}}],"chains":[],"distance":412,"geocodes":{"main":{"latitude":43.64992,"longitude":-79.342227},"roof":{"latitude":43.64992,"longitude":-79.342227}},"link":"/v3/places/4ad8f0abf964a520351621e3","location":{"address":"225 Commissioners

In [29]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = response ['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # objeto JSON

# filtrar columnas
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filtrar la categoría para cada fila
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# limpiar columnas
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

TypeError: 'Response' object is not subscriptable

I couldn't move forward because of the error, which I think is because of the V3 version of foursquare. hope you help me to overcome this error, everyone using foursquare v3